<a href="https://colab.research.google.com/github/BaBa0525/machine-learning/blob/main/HW5/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import package

In [ ]:
import csv
import cv2
import numpy as np
import random
import os

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/ML-HW5
!ls

/content/drive/MyDrive/Colab Notebooks/ML-HW5
captcha-hacker.zip  hw5.ipynb  input  submission.csv


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Check dataset

In [ ]:
# import os

# for dirname, _, filenames in os.walk('./input'):
#     for filename in filenames[:3]:
#         print(os.path.join(dirname, filename))
#     if len(filenames) > 3:
#         print("...")


In [ ]:
TRAIN_PATH = "./input/train"
TEST_PATH = "./input/test"

## Dataset Class

In [ ]:
class Task1Dataset(Dataset):
  def __init__(self, data, root, return_filename=False):
    self.data = [sample for sample in data if sample[0].startswith("task1")]
    self.return_filename = return_filename
    self.root = root

  def __getitem__(self, index):
    filename, label = self.data[index]
    img = cv2.imread(f"{self.root}/{filename}")
    img = cv2.resize(img, (32, 32))
    img = np.mean(img, axis=2)
    if self.return_filename:
      return torch.FloatTensor((img - 128) / 128), filename

    return torch.FloatTensor((img - 128) / 128), int(label)

  def __len__(self):
    return len(self.data)
    

## Model Class

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5),
        nn.LeakyReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=3, out_channels=9, kernel_size=5),
        nn.LeakyReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(9 * 5 * 5, 100),
        nn.LeakyReLU(),
        nn.Linear(100, 50),
        nn.LeakyReLU(),
        nn.Linear(50, 10)
    )

  def forward(self, x):
    # print(f"{x.shape=}")
    # b, h, w = x.shape
    # x = x.view(b, h*w)
    x = x[:, np.newaxis, :]
    print(f"{x.shape=}")
    return self.layers(x)

## Process training data

In [ ]:
train_data = []
val_data = []

with open(f"{TRAIN_PATH}/annotations.csv", newline="") as csvfile:
  for row in csv.reader(csvfile, delimiter=","):
    if random.random() < 0.7:
      train_data.append(row)
    else:
      val_data.append(row)

train_ds = Task1Dataset(train_data, TRAIN_PATH)
train_dl = DataLoader(train_ds, batch_size=500, drop_last=True, shuffle=True)

val_ds = Task1Dataset(val_data, TRAIN_PATH)
val_dl = DataLoader(val_ds, batch_size=500, drop_last=True, shuffle=True)

## Training

In [ ]:
model = Model().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in tqdm(range(100), leave=True):
  print(f"Epoch: [{epoch}]")

  model.train()
  for image, label in train_dl:
    print("---")
    image = image.to(device)
    label = label.to(device)

    pred = model(image)
    loss = loss_fn(pred, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  sample_count = 0
  correct_count = 0

  model.eval()
  for image, label in val_dl:
    image = image.to(device)
    label = label.to(device)

    pred = model(image)
    loss = loss_fn(pred, label)

    pred = torch.argmax(pred, dim=1)

    sample_count += len(image)
    correct_count += (label == pred).sum()

  print(f"accuracy(validation): {correct_count / sample_count}")



  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: [0]
---
x.shape=torch.Size([500, 1, 32, 32])
---
x.shape=torch.Size([500, 1, 32, 32])


  1%|          | 1/100 [05:54<9:44:31, 354.26s/it]

x.shape=torch.Size([500, 1, 32, 32])
accuracy(validation): 0.11800000816583633
Epoch: [1]
---
x.shape=torch.Size([500, 1, 32, 32])
---
x.shape=torch.Size([500, 1, 32, 32])


  2%|▏         | 2/100 [07:18<5:18:51, 195.22s/it]

...

x.shape=torch.Size([500, 1, 32, 32])
accuracy(validation): 0.8220000267028809
Epoch: [98]
---
x.shape=torch.Size([500, 1, 32, 32])
---
x.shape=torch.Size([500, 1, 32, 32])


 99%|█████████▉| 99/100 [12:20<00:02,  2.76s/it]

x.shape=torch.Size([500, 1, 32, 32])
accuracy(validation): 0.8360000252723694
Epoch: [99]
---
x.shape=torch.Size([500, 1, 32, 32])
---
x.shape=torch.Size([500, 1, 32, 32])


100%|██████████| 100/100 [12:23<00:00,  7.43s/it]

x.shape=torch.Size([500, 1, 32, 32])
accuracy(validation): 0.8460000157356262


In [ ]:
# torch.save(model.state_dict(), "task1_model.pt")
print(model.state_dict())


OrderedDict([('layers.0.weight', tensor([[[[-0.0899, -0.0271, -0.0276,  0.0023, -0.0878],
          [-0.0096, -0.0608,  0.1565, -0.1445, -0.0741],
          [-0.0163, -0.0309,  0.0682,  0.0362, -0.0115],
          [-0.0362,  0.0962,  0.2872,  0.1128, -0.1070],
          [ 0.2070,  0.3048,  0.2565,  0.2978,  0.2694]]],


        [[[ 0.0765, -0.0575, -0.1304, -0.0434, -0.1573],
          [-0.0727, -0.0610,  0.0031, -0.2441, -0.2059],
          [ 0.0438, -0.0996,  0.2365, -0.0499, -0.3081],
          [ 0.1063,  0.0428,  0.0502,  0.2179,  0.0092],
          [-0.1050,  0.3127,  0.3362,  0.0311, -0.2202]]],


        [[[ 0.0103, -0.1601, -0.0670,  0.0253,  0.3175],
          [ 0.0317, -0.2246, -0.0945, -0.1124,  0.1035],
          [ 0.2479, -0.0798, -0.2734,  0.1377,  0.1286],
          [ 0.0254, -0.1637, -0.1532,  0.0660, -0.0410],
          [-0.0656,  0.1607, -0.1212, -0.2007,  0.0443]]]], device='cuda:0')), ('layers.0.bias', tensor([-0.0487,  0.0451,  0.1996], device='cuda:0')), ('layers.

## Output prediction

In [ ]:
test_data = []

with open(f"{TEST_PATH}/../sample_submission.csv", newline="") as csvfile:
  for row in csv.reader(csvfile, delimiter=","):
    test_data.append(row)

test_ds = Task1Dataset(test_data, TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=500, num_workers=2, drop_last=False, shuffle=False)

if os.path.exists("submission.csv"):
  fo = open("submission.csv", mode="a", newline="")
  csv_writer = csv.writer(fo)
else:
  fo = open("submission.csv", mode="a", newline="")
  csv_writer = csv.writer(fo)
  csv_writer.writerow(["filename", "label"])

model.eval()
for image, filenames in test_dl:
  image = image.to(device)

  pred = model(image)
  pred = torch.argmax(pred, dim=1)

  for ind, filename in enumerate(filenames):
    if (len(filename) < 3):
      print(filename)
    csv_writer.writerow([filename, str(pred[ind].item())])



x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])
x.shape=torch.Size([500, 1, 32, 32])


In [ ]:
for filename, _ in test_data:
  if filename.startswith("task2") or filename.startswith("task3"):
    # print(filename)
    csv_writer.writerow([filename, 0])

fo.close()

In [ ]:
!ls

captcha-hacker.zip  hw5.ipynb  input  submission.csv
